In [ ]:
def generate_daily_time_series_fn(from_date, to_date, trips_type='all'):
     
    time_series = []
    target_days = generate_days_fn(from_date, to_date)
    for date in tqdm_notebook(target_days):
        df = read_INE_trips_date_fn(date, trips_type)
        df= df.drop(columns=['edad', 'viajes_km','residencia','periodo'])
        group_by_df= df.groupby(['fecha','origen']).sum('viajes')
        group_by_df= group_by_df.reset_index()
        total_trips_final = group_by_df.pivot_table(values='viajes',index='origen',columns='fecha')
        time_series.append(total_trips_final)
    return pd.concat(time_series, axis=1)

In [ ]:
def generate_hourly_time_series_fn(from_date, to_date, trips_type='all'):
     
    time_series = []
    target_days = generate_days_fn(from_date, to_date)
    for date in tqdm_notebook(target_days):
        df = read_INE_trips_date_fn(date, trips_type)
        df= df.drop(columns=['edad', 'viajes_km','residencia'])
        group_by_df= df.groupby(['fecha','origen','periodo']).sum('viajes')
        group_by_df= group_by_df.reset_index()
        group_by_df['fecha_periodo'] = group_by_df['fecha']+'_'+ group_by_df['periodo'].astype(str)
        group_by_df= group_by_df.drop(columns='fecha periodo'.split())
        total_trips_final = group_by_df.pivot_table(values='viajes',index='origen',columns='fecha_periodo')
        time_series.append(total_trips_final)
    return pd.concat(time_series, axis=1)

In [ ]:
def generate_OD_matrix_from_date_fn(date_):
    df = read_INE_trips_date_fn(date_)
    od= pd.pivot_table(df, columns='destino', index='origen', values='viajes', aggfunc=np.sum)
    od = od.fillna(0.0)
    od['total'] = od.sum(axis=1)
    od.loc['total']= od.sum(axis=0)
    return od